In [28]:
import importlib

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

In [29]:
ev_table = MyPyTable()
ev_table.load_from_file("./input_data/Electric_Vehicles_By_County.csv")

rows_to_remove = []
for i in range(len(ev_table.data)):
    if ev_table.data[i][0][-4:] not in ['2019', '2020', '2021']:
        rows_to_remove.append(i)
    elif ev_table.data[i][-2] < 100:
        rows_to_remove.append(i)
ev_table.drop_rows(rows_to_remove)
print(len(ev_table.data))

5075


In [30]:
income_table = MyPyTable()
income_table.load_from_file("./input_data/Income_By_County.csv")

state_abbrev_dict = {"Alabama" : "AL", "Kentucky" : "KY", "Ohio" : "OH", "Alaska" : "AK",
    "Louisiana" : "LA", "Oklahoma" : "OK", "Arizona" : "AZ", "Maine" : "ME", "Oregon" : "OR",
    "Arkansas" : "AR", "Maryland" : "MD", "Pennsylvania" : "PA", "Massachusetts" : "MA",
    "California" : "CA", "Michigan" : "MI", "Rhode Island" : "RI", "Colorado" : "CO",
    "Minnesota" : "MN", "South Carolina" : "SC", "Connecticut" : "CT", "Mississippi" : "MS",
    "South Dakota" : "SD", "Delaware" : "DE", "Missouri" : "MO", "Tennessee" : "TN",
    "District of Columbia" : "DC", "Montana" : "MT", "Texas" : "TX", "Florida" : "FL",
    "Nebraska" : "NE", "Georgia" : "GA", "Nevada" : "NV", "Utah" : "UT", "New Hampshire" : "NH",
    "Vermont" : "VT", "Hawaii" : "HI", "New Jersey" : "NJ", "Virginia" : "VA", "Idaho" : "ID",
    "New Mexico" : "NM", "Illinois" : "IL", "New York" : "NY", "Washington" : "WA",
    "Indiana" : "IN", "North Carolina" : "NC", "West Virginia" : "WV", "Iowa" : "IA",
    "North Dakota" : "ND", "Wisconsin" : "WI", "Kansas" : "KS", "Wyoming" : "WY"}

new_col = []
rows_to_remove2 = []
current_state = None
for i in range(len(income_table.data)):
    if income_table.data[i][0] == "separator":
        rows_to_remove2.append(i)
    elif income_table.data[i - 1][0] == "separator":
        current_state = state_abbrev_dict[income_table.data[i][0]]
        rows_to_remove2.append(i)
    else:
        new_col.append(current_state)

income_table.drop_rows(rows_to_remove2)
income_table.add_column("State", new_col)
print(income_table.data[:25])

[['Autauga', '42,846', '45,248', '48,347', 11.0, 5.6, 6.8, 60.0, '', 'AL'], ['Baldwin', '48,380', '51,348', '54,659', 4.0, 6.1, 6.4, 63.0, '', 'AL'], ['Barbour', '34,870', '37,120', '40,428', 54.0, 6.5, 8.9, 33.0, '', 'AL'], ['Bibb', '31,800', '34,598', '36,892', 66.0, 8.8, 6.6, 62.0, '', 'AL'], ['Blount', '36,542', '38,351', '42,634', 39.0, 5.0, 11.2, 10.0, '', 'AL'], ['Bullock', '27,192', '30,429', '33,267', 67.0, 11.9, 9.3, 27.0, '', 'AL'], ['Butler', '39,254', '41,151', '46,050', 20.0, 4.8, 11.9, 7.0, '', 'AL'], ['Calhoun', '36,869', '39,546', '42,621', 40.0, 7.3, 7.8, 48.0, '', 'AL'], ['Chambers', '33,682', '36,299', '39,728', 58.0, 7.8, 9.4, 26.0, '', 'AL'], ['Cherokee', '37,751', '39,416', '43,719', 32.0, 4.4, 10.9, 11.0, '', 'AL'], ['Chilton', '36,195', '38,551', '41,748', 44.0, 6.5, 8.3, 40.0, '', 'AL'], ['Choctaw', '38,651', '41,413', '44,241', 29.0, 7.1, 6.8, 61.0, '', 'AL'], ['Clarke', '37,426', '41,862', '45,232', 23.0, 11.9, 8.1, 43.0, '', 'AL'], ['Clay', '32,059', '34,66

In [31]:
joined_table = ev_table.perform_inner_join(income_table, ["County", "State"])

In [32]:
print(ev_table.column_names)
print(income_table.column_names)
for i in range(len(ev_table.column_names)):
    if ev_table.column_names[i] in income_table.column_names:
        print(ev_table.column_names[i])
print(joined_table.data[:25])
print(len(joined_table.data))

['Date', 'County', 'State', 'Vehicle Primary Use', 'Battery Electric Vehicles (BEVs)', 'Plug-In Hybrid Electric Vehicles (PHEVs)', 'Electric Vehicle (EV) Total', 'Non-Electric Vehicle Total', 'Total Vehicles', 'Percent Electric Vehicles']
['County', ' Dollars 2019', ' Dollars 2020', ' Dollars 2021', ' Dollars Rank in state', ' Percent change 2020', ' Percent change 2021', ' Percent change Rank in state', 'State']
County
State
[]
0
